# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32)
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp=hgh/lda/si-q4)
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic(1)
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               Hartree(1)
          

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.120118e+02     1.420426e+00
 * time: 0.6817131042480469
     1     1.010130e+01     8.712026e-01
 * time: 2.0104241371154785
     2    -1.249473e+01     9.608107e-01
 * time: 2.6683170795440674
     3    -3.448065e+01     7.452178e-01
 * time: 3.6437649726867676
     4    -4.805041e+01     5.712158e-01
 * time: 4.6558921337127686
     5    -5.690955e+01     1.714700e-01
 * time: 5.640860080718994
     6    -5.971687e+01     1.092351e-01
 * time: 6.296607971191406
     7    -6.084303e+01     4.552844e-02
 * time: 6.935544013977051
     8    -6.128101e+01     5.931990e-02
 * time: 7.572005987167358
     9    -6.159389e+01     3.950439e-02
 * time: 8.208033084869385
    10    -6.182012e+01     2.579984e-02
 * time: 8.855574131011963
    11    -6.199242e+01     2.334469e-02
 * time: 9.510976076126099
    12    -6.205645e+01     1.924282e-02
 * time: 10.14306092262268
    13    -6.212314e+01     1.220343e-02
 * time: 10.77394413948059
  

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671075
    AtomicLocal         -18.8557703
    AtomicNonlocal      14.8522656
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485383 
    Xc                  -19.3336825

    total               -62.261666461722